# Final Evaluation - Thesis Report

 - Evaluate on all images in a scene


In [1]:
# Evaluation variables

MASK_NAME = 'ViT_masks'



############# View-Fusion evaluation 
### Mask2Former

# None

# Majority vote

# Transformer

# DeepSet

### ViT-Adapter

# None

# Majority vote

# # Transformer - MVFusion_orig
# checkpoint_dir = '/home/fsun/DeepViewAgg/outputs/MVFusion_orig'

# DeepSet - DeepSetAttention (check model/config correctness)
checkpoint_dir = '/home/fsun/DeepViewAgg/outputs/2023-01-17/18-49-35'
dataset_config = 'segmentation/multimodal/Feng/scannet-neucon-smallres-m2f-no-mix3d.yaml'   
models_config = 'segmentation/multimodal/Feng/view_selection_experiment'    # model family
model_name = 'DeepSetAttention'                       # specific model


############# View-Fusion + 3D network evaluation


### Mask2Former

# Transformer - MVFusion_3D_small_6views
# CHECKPOINT_DIR = "/home/fsun/DeepViewAgg/outputs/MVFusion_3D_6_views_m2f_masks"

# DeepSet - Deepset_3D
# checkpoint_dir = "/home/fsun/DeepViewAgg/outputs/2023-02-05/23-15-04"

### ViT-Adapter

# Transformer - MVFusion_3D_small_6views
# checkpoint_dir = '/home/fsun/DeepViewAgg/outputs/ViT_masks_3rd_run' # 3rd run

# DeepSet - Deepset_3D
# checkpoint_dir = "/home/fsun/DeepViewAgg/outputs/2023-01-23/12-57-16"


# dataset_config = 'segmentation/multimodal/Feng/scannet-neucon-smallres-m2f.yaml'   
# models_config = 'segmentation/multimodal/Feng/mvfusion'    # model family
# model_name = 'MVFusion_3D_small_6views'                       # specific model

In [2]:
# # Set your dataset root directory, where the data was/will be downloaded
# DATA_ROOT = '/scratch-shared/fsun/dvata'


# overrides = [
#     'task=segmentation',
#     f'data={dataset_config}',
#     f'models={models_config}',
#     f'model_name={model_name}',
#     f'data.dataroot={DATA_ROOT}',
# ]

# cfg = hydra_read(overrides)
# OmegaConf.set_struct(cfg, False)  # This allows getattr and hasattr methods to function correctly
# cfg.data.load_m2f_masks = True   # load Mask2Former predicted masks
# cfg.data.m2f_preds_dirname = MASK_NAME
# cfg.data.n_views = cfg.models[model_name].backbone.transformer.n_views
# print(cfg.data.n_views)

# # Dataset instantiation
# start = time()
# dataset = ScannetDatasetMM(cfg.data)
# # print(dataset)|
# print(f"Time = {time() - start:0.1f} sec.")

In [3]:
# from torch_points3d.models.model_factory import instantiate_model

# # Create the model
# print(f"Creating model: {cfg.model_name}")
# model = instantiate_model(cfg, dataset)
# # print(model)

# # Load the checkpoint and recover the 'best_miou' model weights
# checkpoint = torch.load(f'{CHECKPOINT_DIR}/{model_name}.pt', map_location='cpu')
# model.load_state_dict_with_same_shape(checkpoint['models']['best_miou'], strict=False)

# # Prepare the model for training
# model = model.cuda()
# print('Model loaded')

In [4]:
# import pandas as pd
# pd.set_option('display.max_rows', 50)

# # Create validation loader
# dataset.create_dataloaders(
#     model,
#     1,
#     False,
#     17,
#     False,
#     train_only=False,
#     val_only=True,
#     test_batch_size=1
# )

# mapping_idx_to_scan_names = getattr(dataset.val_dataset, "MAPPING_IDX_TO_SCAN_{}_NAMES".format(dataset.val_dataset.split.upper()))


In [1]:
# Uncomment to use autoreload
%load_ext autoreload
%autoreload 2

import os
import os.path as osp
import sys
import torch
import numpy as np
from time import time
from omegaconf import OmegaConf
start = time()
import warnings
warnings.filterwarnings('ignore')

# torch.cuda.set_device(I_GPU)
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)

from torch_points3d.utils.config import hydra_read
from torch_geometric.data import Data
from torch_points3d.core.multimodal.data import MMData, MMBatch
from torch_points3d.visualization.multimodal_data import visualize_mm_data
from torch_points3d.core.multimodal.image import SameSettingImageData, ImageData
from torch_points3d.datasets.segmentation.multimodal.scannet import ScannetDatasetMM
from torch_points3d.datasets.segmentation.scannet import CLASS_COLORS, CLASS_NAMES, CLASS_LABELS
from torch_points3d.metrics.segmentation_tracker import SegmentationTracker
from torch_points3d.datasets.segmentation import IGNORE_LABEL
from torch_points3d.metrics.scannet_segmentation_tracker import ScannetSegmentationTracker
from torch_points3d.metrics.colored_tqdm import Coloredtqdm as Ctq


from PIL import Image

import matplotlib.pyplot as plt 

%matplotlib inline

CLASS_COLORS[0] = (174.0, 199.0, 232.0)
CLASS_COLORS[-1] = (0, 0, 0)
import plotly.io as pio

#pio.renderers.default = 'jupyterlab'        # for local notebook
pio.renderers.default = 'iframe_connected'  # for remote notebook. Other working (but seemingly slower) options are: 'sphinx_gallery' and 'iframe'

MMData debug() function changed, please uncomment the 3rd assert line when doing inference without M2F features!


In [2]:
# Functions for evaluation

def get_seen_points(mm_data, return_mask=False):
    ### Select seen points
    csr_idx = mm_data.modalities['image'][0].view_csr_indexing
    
    seen_mask = (csr_idx[1:] > csr_idx[:-1])
    if return_mask:
        return mm_data[seen_mask], seen_mask
    else:
        return mm_data[seen_mask]

def get_mode_pred(data):
    pixel_validity = data.data.mvfusion_input[:, :, 0].bool()
    mv_preds = data.data.mvfusion_input[:, :, -1].long()
            
    valid_m2f_feats = []
    for i in range(len(mv_preds)):
        valid_m2f_feats.append(mv_preds[i][pixel_validity[i]])

    mode_preds = []
    for m2feats_of_seen_point in valid_m2f_feats:
        mode_preds.append(torch.mode(m2feats_of_seen_point.squeeze(), dim=0)[0])
    mode_preds = torch.stack(mode_preds, dim=0)
        
    return mode_preds

def get_normalized_entropy(labels):
    counts = torch.unique(labels, return_counts=True)[1]
    
    pk = counts / counts.sum()
    len_pk = torch.tensor(len(pk))
    if len_pk == 1:
        normalized_entropy = 0.
    else:
        normalized_entropy = -sum(pk * torch.log2(pk)) / torch.log2(len_pk)
    return normalized_entropy
        
def get_semantic_image_from_camera(dataset, scene, mesh_triangles, intrinsic, extrinsic, class_id_faces, im_size=(480, 640)):
    """
    Returns the back-projected semantic label image given camera parameters and (semantic) mesh.
    """
    
    # Initialize rays for given camera
    rays = o3d.t.geometry.RaycastingScene.create_rays_pinhole(
        intrinsic_matrix=intrinsic,
        extrinsic_matrix=extrinsic,
        width_px=im_size[1],
        height_px=im_size[0],
    )

    # Get result
    ans = scene.cast_rays(rays)

    primitive_ids = ans['primitive_ids'].numpy()
    primitive_uvs = ans['primitive_uvs'].numpy()

    # Select the closest vertex for each valid face in the projected mesh
    valid_mask = primitive_ids != scene.INVALID_ID

    # https://stackoverflow.com/questions/45212949/vertex-of-a-3d-triangle-that-is-closest-to-a-point-given-barycentric-parameter-o
    w_coords = (1 - primitive_uvs[:, :, 0][valid_mask] - primitive_uvs[:, :, 1][valid_mask])
    barycentric_coords = np.concatenate((w_coords[:, None], primitive_uvs[valid_mask]), axis=-1)

    selected_vertex_idx = np.argmax(barycentric_coords, axis=-1)

    contained_mesh_triangles = mesh_triangles[primitive_ids[valid_mask]]
    closest_mesh_vertices = contained_mesh_triangles[range(len(barycentric_coords)), selected_vertex_idx]
    
    # Map mesh vertices to semantic label
    labels = class_id_faces[closest_mesh_vertices]
    # Remap to [0 ; num_labels - 1]
    labels = dataset.val_dataset._remap_labels(torch.tensor(labels))

    # Visualize back-projection
    image = torch.ones(im_size, dtype=torch.long) * -1
    image[valid_mask] = labels


    # NN interpolation at invalid pixels          
    nearest_neighbor = scipy.ndimage.morphology.distance_transform_edt(
        image==-1, return_distances=False, return_indices=True)    

    image = image[nearest_neighbor].numpy()
    return image

def read_axis_align_matrix(filename):
    lines = open(filename).readlines()
    axis_align_matrix = None
    for line in lines:
        if "axisAlignment" in line:
            axis_align_matrix = torch.Tensor([float(x) for x in line.rstrip().strip("axisAlignment = ").split(" ")]).reshape((4, 4))
            break
    return axis_align_matrix

def save_semantic_prediction_as_txt(tracker, model_name, mask_name):
    orginal_class_ids = np.asarray(tracker._dataset.train_dataset.valid_class_idx)
    path_to_submission = tracker._dataset.path_to_submission
    
    path_to_submission = osp.join(path_to_submission, model_name, mask_name)
    if not osp.exists(path_to_submission):
        os.makedirs(path_to_submission)
    
    for scan_id in tracker._full_preds:
        full_pred = tracker._full_preds[scan_id].cpu().numpy().astype(np.int8)
        full_pred = orginal_class_ids[full_pred]  # remap labels to original labels between 0 and 40
        scan_name = tracker._raw_datas[scan_id].scan_name
        path_file = osp.join(path_to_submission, "{}.txt".format(scan_name))
        
        np.savetxt(path_file, full_pred, delimiter="/n", fmt="%d")
        
    return path_to_submission
        
        


In [24]:
import os
import os.path as osp
import copy
import torch
import hydra
import time
import logging
import scipy.ndimage
import numpy as np
from PIL import Image
import open3d as o3d

# Import building function for model and dataset
from torch_points3d.datasets.dataset_factory import instantiate_dataset
from torch_points3d.models.model_factory import instantiate_model

# Import BaseModel / BaseDataset for type checking
from torch_points3d.models.base_model import BaseModel
from torch_points3d.datasets.base_dataset import BaseDataset

# Import from metrics
from torch_points3d.metrics.base_tracker import BaseTracker
from torch_points3d.metrics.colored_tqdm import Coloredtqdm as Ctq
from torch_points3d.metrics.model_checkpoint import ModelCheckpoint

# Utils import
from torch_points3d.utils.colors import COLORS
from torch_points3d.utils.wandb_utils import Wandb
from torch_points3d.utils.config import getattr_recursive
from torch_points3d.visualization import Visualizer
from torch_points3d.core.data_transform.transforms import PointcloudMerge
from torch_points3d.datasets.segmentation.scannet import CLASS_COLORS, CLASS_NAMES, CLASS_LABELS





log = logging.getLogger(__name__)


class Evaluator():
    
    def __init__(self, cfg, checkpoint_dir, early_break=False):
        self.scans_dir = "/scratch-shared/fsun/data/scannet/scans"

#         self._dataset = dataset
        self._cfg = cfg
    
        if self._cfg.model_name in ['MVFusion_orig', 'DeepSetAttention']:
            self.no_3d = True
        else:
            self.no_3d = False
            
        print("self.no_3d == ", self.no_3d)
    
        self.wandb_log = False
        self.tensorboard_log = False
        
        self.early_break = early_break

        
#         # Create the model
#         print(f"Creating model: {self._cfg.model_name}")
#         model = instantiate_model(self._cfg, self._dataset)

#         # Load the checkpoint and recover the 'best_miou' model weights
#         checkpoint = torch.load(f'{checkpoint_dir}/{self._cfg.model_name}.pt', map_location='cpu')
#         model.load_state_dict_with_same_shape(checkpoint['models']['best_miou'], strict=False)

#         # Prepare the model for training
#         self._model = model.cuda()
#         self._device = self._model.device

#         print(f"Device is {self._device}")
        
        
        # Enable CUDNN BACKEND
        torch.backends.cudnn.enabled = self.enable_cudnn

        if not self.has_training:
            self._cfg.training = self._cfg
            resume = bool(self._cfg.checkpoint_dir)
        else:
            resume = bool(self._cfg.training.checkpoint_dir)

        # Get device
        if self._cfg.training.cuda > -1 and torch.cuda.is_available():
            device = "cuda"
            torch.cuda.set_device(self._cfg.training.cuda)
        else:
            device = "cpu"
        self._device = torch.device(device)
        log.info("DEVICE : {}".format(self._device))

        
        # Start Wandb if public
        if self.wandb_log:
            Wandb.launch(
                self._cfg, self._cfg.training.wandb.public and self.wandb_log)

        # Checkpoint
        self._checkpoint: ModelCheckpoint = ModelCheckpoint(
            checkpoint_dir, self._cfg.model_name,
            "", run_config=self._cfg, resume=resume)
            
        # Recover the merged config from Checkpoint
        self._cfg = self._checkpoint.run_config

        # Update dataset config with number of views for the MVFusion Transformer
        model_config = getattr(self._cfg.models, self._cfg.model_name, None)
        self._cfg.data.n_views = model_config.backbone.transformer.n_views
                
        # Create model and datasets
        if not self._checkpoint.is_empty:
            self._dataset: BaseDataset = instantiate_dataset(self._cfg.data)
            self._model: BaseModel = self._checkpoint.create_model(
                self._dataset, weight_name="best_miou")
        else:
            log.warning("Checkpoint is empty or model cannot be instantiated from given checkpoint.")
#         else:
#             self._dataset: BaseDataset = instantiate_dataset(self._cfg.data)
#             self._model: BaseModel = instantiate_model(
#                 copy.deepcopy(self._cfg), self._dataset)
#             self._model.instantiate_optimizers(self._cfg, "cuda" in device)
#             self._model.set_pretrained_weights()
#             print("self._checkpoint.data_config: ", self._checkpoint.data_config)
#             if not self._checkpoint.validate(self._dataset.used_properties):
#                 log.warning(
#                     "The model will not be able to be used from pretrained "
#                     "weights without the corresponding dataset. Current "
#                     "properties are {}".format(self._dataset.used_properties))
                
            
#             return None
                
        self._checkpoint.dataset_properties = self._dataset.used_properties

        log.info(self._model)

        self._model.log_optimizers()
        log.info("Model size = %i", sum(param.numel() for param in self._model.parameters() if param.requires_grad))

        # Set dataloaders
        self._dataset.create_dataloaders(
            self._model,
            self._cfg.training.batch_size,
            self._cfg.training.shuffle,
            self._cfg.training.num_workers,
            self.precompute_multi_scale,
            train_only=False,
            val_only=True,
            test_batch_size=1
        )
        
        log.info(self._dataset)

        # Verify attributes in dataset
        self._model.verify_data(self._dataset.train_dataset[0])

        # Choose selection stage
        selection_stage = getattr(self._cfg, "selection_stage", "")
        self._checkpoint.selection_stage = self._dataset.resolve_saving_stage(
            selection_stage)
        
        self.mapping_idx_to_scan_names = getattr(self._dataset.val_dataset, "MAPPING_IDX_TO_SCAN_{}_NAMES".format(self._dataset.val_dataset.split.upper()))

        
    
    def eval_all_metrics(self, stage_name=""):
        self._is_training = False
        
        self._tracker_refined: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
        self._tracker_refined_seen_points: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)            
                        
        if self._dataset.has_val_loader:
            if not stage_name or stage_name == "val":
                print("Evaluating on validation set")
                self._test_refined(epoch=1, stage_name="val")   
                
        # Upscale predictions containing all points to 0.01 voxel size and save point cloud predictions   
        self._tracker_refined.finalise(full_res=True)
        path_to_submission = save_semantic_prediction_as_txt(
            self._tracker_refined, self._cfg.model_name, self._dataset.val_dataset.m2f_preds_dirname)
        
        # Back-project semantic mesh (from pcd) to 2D images given the maximum number of views per scene, and save.
        # Skips this step if refined images already exist for given model and mask
        self.mesh_to_image(self._cfg, self._dataset, path_to_submission, self.scans_dir, save_output='if_not_exists') 
        
        # Evaluate 2D semantic segmentation
        self._tracker_refined_2d_iou: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
        
        self._evaluate_2d_iou()
        
        # Evaluate 2D cross-view consistency
        # Temporary solution to evaluate both 2D input masks and 2D refined masks for Cross-view Consistency:
        # load refined masks into gt_mask attribute
        self._dataset.val_dataset.gt_dir_name = osp.join(f"{self._dataset.val_dataset.m2f_preds_dirname}_refined", 
                                                         self._cfg.model_name)
        self._evaluate_2d_CC()        
        
        # Evaluate 2D temporal consistency
        self.tracker_TC_baseline: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
        self.tracker_TC_refined: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
        self._evaluate_2d_TC()   
        
        
        
    def _evaluate_2d_TC(self):
        from mmflow.apis import init_model, inference_model
        from mmflow.datasets import visualize_flow, write_flow
        import mmcv
        from mmflow.ops import Warp

        # Specify the path to model config and checkpoint file
        config_file = '/home/fsun/DeepViewAgg/flow/configs/flownet2_8x1_sfine_flyingthings3d_subset_384x768.py'
        checkpoint_file = '/home/fsun/DeepViewAgg/flow/pretrained/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth'

        # build the model from a config file and a checkpoint file
        flow_model = init_model(config_file, checkpoint_file, device='cuda:0')

        warp = Warp(mode='nearest',
                    padding_mode='zeros',
                    align_corners=False,
                    use_mask=True).cuda()
        
    
        mask_foldername = self._dataset.val_dataset.m2f_preds_dirname
        refined_mask_foldername = osp.join(f"{mask_foldername}_refined", self._cfg.model_name)
        mask_scans_dir = '/scratch-shared/fsun/data/scannet/scans'
        

        self.tracker_TC_baseline.reset(stage='val')
        self.tracker_TC_refined.reset(stage='val')

        scan_ids = list(self.mapping_idx_to_scan_names.values())

        scans_dir = "/scratch-shared/fsun/data/scannet/scans"

        for scan_id in Ctq(scan_ids):
            scan_dir = osp.join(scans_dir, scan_id)

            mask_scan_dir = osp.join(mask_scans_dir, scan_id)

            mask_dir = osp.join(mask_scan_dir, mask_foldername)
            refined_mask_dir = osp.join(scan_dir, refined_mask_foldername)

            refined_mask_names = os.listdir(refined_mask_dir)
            refined_mask_names = sorted(refined_mask_names, key=lambda i: int(os.path.splitext(os.path.basename(i))[0]))

            color_im_dir = osp.join(scan_dir, 'color_resized')

            # Loop over all pairs of consecutive images
            for i in range(len(refined_mask_names) - 1):
                cur_frame_name, next_frame_name = refined_mask_names[i], refined_mask_names[i+1]
                cur_frame_id, next_frame_id = int(refined_mask_names[i].split(".")[0]), int(refined_mask_names[i+1].split(".")[0])

                # Skip if jump in frames is too big
                if next_frame_id - cur_frame_id > 50:
                    continue

                cur_color_p, next_color_p = osp.join(color_im_dir, cur_frame_name), osp.join(color_im_dir, next_frame_name)
                cur_color_im, next_color_im = Image.open(cur_color_p), Image.open(next_color_p)

                result = inference_model(flow_model, next_color_p, cur_color_p)
                flow_map = torch.tensor(result).permute(2, 0, 1).unsqueeze(0)

                ################# 2D input masks
                seg_im_p1 = osp.join(mask_dir, cur_frame_name)
                seg_im_p2 = osp.join(mask_dir, next_frame_name)
                seg_im1 = np.asarray(Image.open(seg_im_p1)) 
                seg_im2 = np.asarray(Image.open(seg_im_p2)).astype(np.int) - 1   # Adjust labels

                # warping im1 to im2
                seg_im1_semantic = torch.tensor(seg_im1).unsqueeze(0).unsqueeze(0).float()
                seg_im_warped = warp(seg_im1_semantic, flow_map).permute(0, 2, 3, 1)[0].squeeze() - 1    # Adjust labels

                # take im2 as 'pseudo gt' for temporal consistency. Thus, invalidly warped pixels should be ignored
                # by setting the corresponding gt label to the IGNORE_LABEL
                seg_im2[seg_im_warped == -1] = IGNORE_LABEL
                self.tracker_TC_baseline.track(pred_labels=seg_im_warped.long(), gt_labels=seg_im2, model=None)

                ################ 2D refined masks
                seg_im_p1 = osp.join(refined_mask_dir, cur_frame_name)
                seg_im_p2 = osp.join(refined_mask_dir, next_frame_name)
                seg_im1 = np.asarray(Image.open(seg_im_p1))  + 1 # Adjust labels for flow warping (invalid pixels will be set to 0)

                # Resize in case
                seg_im2 = Image.open(seg_im_p2).resize((640, 480), resample=0)
                seg_im2 = np.asarray(seg_im2).astype(np.int) 

                # warping im1 to im2
                seg_im1_semantic = torch.tensor(seg_im1).unsqueeze(0).unsqueeze(0).float()
                seg_im_warped = warp(seg_im1_semantic, flow_map).permute(0, 2, 3, 1)[0].squeeze() - 1 # Adjust labels back to normal

                # take im2 as 'pseudo gt' for temporal consistency. Thus, invalidly warped pixels should be ignored
                # by setting the corresponding gt label to the IGNORE_LABEL
                seg_im2[seg_im_warped == -1] = IGNORE_LABEL
                self.tracker_TC_refined.track(pred_labels=seg_im_warped.long(), gt_labels=seg_im2, model=None)

            if self.early_break:
                break
            
        print("Temporal Consistency Scores measured over all validation scenes, using all image views")
        print("Input 2D", self.tracker_TC_baseline.get_metrics())
        print(self.tracker_TC_baseline._miou_per_class)   
        print("Refined 2D ", self.tracker_TC_refined.get_metrics())
        print(self.tracker_TC_refined._miou_per_class)   

    def _evaluate_2d_CC(self):
        
        temp_tracker: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
                
        # Load instance labels for cross-view consistency evaluation
        self._dataset.val_dataset.load_instance_labels = True
                
        del self._dataset._val_loader
        
        # Re-create dataloader
        self._dataset.create_dataloaders(
            self._model,
            self._cfg.training.batch_size,
            self._cfg.training.shuffle,
            0,#self._cfg.training.num_workers,
            self.precompute_multi_scale,
            train_only=False,
            val_only=True,
            test_batch_size=1
        )
                
        self.instance_count = torch.zeros(2, 20)
        self.cum_n_entropy = torch.zeros(2, 20)

        self._model.eval()
            
        with Ctq(self._dataset.val_dataloader) as tq_loader:
            for batch in tq_loader:
                
                with torch.no_grad():
                    self._model.set_input(batch, self._device)
                    
                    with torch.cuda.amp.autocast(enabled=self._model.is_mixed_precision()):
                        self._model.forward(epoch=1)

                    batch.data.pred = self._model.output.detach().cpu().argmax(1)

                    batch = get_seen_points(batch)


                    temp_tracker.track(model=None, pred_labels=batch.data.pred, gt_labels=batch.data.y)

                    # Accumulate entropy of seen points
                    self.add_entropy_to_accumulator(batch)
                    
                if self.early_break:
                    break
            
        print("3D seen points metrics")
        print(temp_tracker.get_metrics())
        
        self.cum_n_entropy = self.cum_n_entropy / (self.instance_count + 1e-8)
        self.crossview_consistency = (1 - self.cum_n_entropy) * 100       
        print("2D cross-view consistency (CC) for input masks")
        print(self.crossview_consistency[0])
        print("Mean: ", self.crossview_consistency[0].mean())

        print("2D cross-view consistency (CC) for refined masks")
        print(self.crossview_consistency[1])
        print("Mean: ", self.crossview_consistency[1].mean())

        
                
    def add_entropy_to_accumulator(self, mm_data):
        for instance_id in mm_data.data['instance_labels'].unique():

            if instance_id == 0:
                continue

            instance_mask = mm_data.data['instance_labels'] == instance_id
            instance = mm_data[instance_mask]
            instance_class = instance.y[0]

            # Skip invalid semantic class
            if instance_class == IGNORE_LABEL:
                continue

            # Track all per-point predictions of active views for current instance. Some points have more 
            # predictions than others.
            input_preds = instance.modalities['image'][0].get_mapped_m2f_features().squeeze()
            output_preds = instance.modalities['image'][0].get_mapped_gt_labels().squeeze() + 1   # +1 label offset

            input_n_entropy = get_normalized_entropy(input_preds)
            output_n_entropy = get_normalized_entropy(output_preds)

            self.cum_n_entropy[0, instance_class] += input_n_entropy
            self.cum_n_entropy[1, instance_class] += output_n_entropy
            self.instance_count[:, instance_class] += 1

    def mesh_to_image(self, cfg, dataset, path_to_submission, scans_dir, save_output='if_not_exists'):
        # User input
        output_image_size = (480, 640)
        preprocessed_2d_data_dir = "/scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/processed/processed_2d_val"

        ########################################################################################################################
        input_mask_name = cfg.data.m2f_preds_dirname
        scan_names = list(dataset.val_dataset.MAPPING_IDX_TO_SCAN_VAL_NAMES.values())

#         path_to_submission = osp.join(path_to_submission, cfg.model_name, input_mask_name)

        for scan_name in Ctq(scan_names):
            # Output folder location
            refined_mask_dir = osp.join(scans_dir, scan_name, f"{input_mask_name}_refined", f"{cfg.model_name}")

            if not osp.exists(refined_mask_dir):
                os.makedirs(refined_mask_dir)

#             # Skip this step if output folder already contains masks
#             if len(os.listdir(refined_mask_dir)) < 10:

            # Load data
            mesh = o3d.io.read_triangle_mesh(f"{scans_dir}/{scan_name}/{scan_name}_vh_clean_2.ply")
            mesh_triangles = np.asarray(mesh.triangles)
            mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)

            # Load predicted class label per vertex
            class_id_faces = np.loadtxt(f"{path_to_submission}/{scan_name}.txt").astype(int)

            # Camera parameters
            intrinsic = np.loadtxt(f"{scans_dir}/{scan_name}/sens/intrinsic/intrinsic_depth.txt")[:3, :3]
            images = torch.load(f"{preprocessed_2d_data_dir}/{scan_name}.pt")

            # Undo axis alignment for extrinsics  
            axis_align_matrix_path = osp.join(scans_dir, scan_name, scan_name + '.txt')
            axis_align_matrix = read_axis_align_matrix(axis_align_matrix_path)
            inv = torch.linalg.inv(axis_align_matrix.T)
            images.extrinsic = inv.T  @ images.extrinsic        

            # Make world-to-camera
            extrinsics = torch.linalg.inv(images.extrinsic).numpy()
            image_names = [osp.splitext(osp.basename(x))[0] for x in images.path]

            # Raycasting
            scene = o3d.t.geometry.RaycastingScene()
            scene.add_triangles(mesh)

            for i in range(len(image_names)):
                image = get_semantic_image_from_camera(dataset=dataset, scene=scene, mesh_triangles=mesh_triangles, intrinsic=intrinsic,
                                                       extrinsic=extrinsics[i], 
                                                       class_id_faces=class_id_faces, im_size=output_image_size)


                # Save refined prediction (backprojected from mesh + interpolated missing pixels)
                image = Image.fromarray(image.astype(np.uint8), 'L')
                im_save_path = osp.join(refined_mask_dir, image_names[i] + '.png')
                image.save(im_save_path)   

#             else:
#                 print("Output directory already exists and contains >10 masks!")

            if self.early_break:
                break
        
    def _evaluate_2d_iou(self):
        input_mask_name = self._cfg.data.m2f_preds_dirname

        scan_names = list(self._dataset.val_dataset.MAPPING_IDX_TO_SCAN_VAL_NAMES.values())
        
        self._tracker_refined_2d_iou.reset(stage="val")

        with Ctq(self._dataset.val_dataloader) as tq_loader:
            for batch in tq_loader:
                scan_name = self.mapping_idx_to_scan_names[batch.id_scan.item()]

                gt_dir = osp.join(self.scans_dir, scan_name, 'label-filt-scannet20')
                mask_dir = osp.join(self.scans_dir, scan_name, f"{input_mask_name}_refined", f"{cfg.model_name}")

                im_names = [osp.basename(x) for x in batch.modalities['image'][0].m2f_pred_mask_path]

                for im in im_names:
                    gt = Image.open(osp.join(gt_dir, im))
                    mask = Image.open(osp.join(mask_dir, im))
                    gt = gt.resize((640, 480), 0)

                    gt = np.asarray(gt).astype(float) - 1
                    mask = np.asarray(mask)

                    self._tracker_refined_2d_iou.track(pred_labels=mask, gt_labels=gt, model=None)
                    
                if self.early_break:
                    break

        print("2D evaluation of refined masks")
        print(self._tracker_refined_2d_iou.get_metrics())
        print(self._tracker_refined_2d_iou._miou_per_class)       
            
    def _test_refined(self, epoch, stage_name: str):

        loaders = [self._dataset.val_dataloader]

        self._model.eval()
            
        for loader in loaders:
            print("Input mask name: ", loader.dataset.m2f_preds_dirname)
            
            stage_name = loader.dataset.name
            self._tracker_refined.reset(stage_name)
            self._tracker_refined_seen_points.reset(stage_name)
            
            with Ctq(loader) as tq_loader:
                for data in tq_loader:
                    with torch.no_grad():
                        if self.no_3d:
                            data, seen_mask = get_seen_points(data, return_mask=True)
                            self._model.set_input(data, self._device)
                        else:
                            self._model.set_input(data, self._device)
                            
                        with torch.cuda.amp.autocast(enabled=self._model.is_mixed_precision()):
                            self._model.forward(epoch=epoch)
                            
                            
                        data.data.pred = self._model.output.detach().cpu().argmax(1)

                        
                        # 3D mIoU, all points
                        self._tracker_refined.track(self._model, full_res=True, data=data)

                        if not self.no_3d:
                            # 3D mIoU, seen points
                            data = get_seen_points(data)
                        self._tracker_refined_seen_points.track(pred_labels=data.data.pred, gt_labels=data.data.y, model=None)
        

                    tq_loader.set_postfix(**self._tracker_refined.get_metrics())
                    
                    if self.early_break:
                        break

            print("Evaluated scores for 3D semantic segmentation on all points: ")
            
            print("--- Refined 3D ---")
            print(self._tracker_refined.get_metrics())
            print(self._tracker_refined._miou_per_class)
            
            print("Evaluated scores for 3D semantic segmentation on seen points: ")
    
            print("--- Refined 3D ---")
            print(self._tracker_refined_seen_points.get_metrics())
            print(self._tracker_refined_seen_points._miou_per_class)
            
                            
        
    def eval_baseline(self, stage_name=""):
        self._is_training = False
        
        self._tracker_baseline: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
            
        self._tracker_baseline_2d: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
                        
        epoch = 1

        if self._dataset.has_val_loader:
            if not stage_name or stage_name == "val":
                self._test_baseline(epoch, "val")   

    @property
    def has_training(self):
        return getattr(self._cfg, "training", None)
    
    @property
    def enable_cudnn(self):
        if self.has_training:
            return getattr(self._cfg.training, "enable_cudnn", True)
        else:
            return getattr(self._cfg, "enable_cudnn", True)

    @property
    def precompute_multi_scale(self):
        if not self.has_training:
            return self._model.conv_type == "PARTIAL_DENSE" and getattr(self._cfg.training, "precompute_multi_scale", False)
        else:
            return self._model.conv_type == "PARTIAL_DENSE" and getattr(self._cfg, "precompute_multi_scale", False)

In [25]:
# ### Note: set over to Python file with argparse from script file
# MASK_NAME = 'ViT_masks'

# dataset_config = 'segmentation/multimodal/Feng/scannet-neucon-smallres-m2f-allviews.yaml'   
# models_config = 'segmentation/multimodal/Feng/mvfusion'    # model family
# model_name = 'MVFusion_3D_small_6views'                       # specific model

# overrides = [
#     'task=segmentation',
#     f'data={dataset_config}',
#     f'models={models_config}',
#     f'model_name={model_name}',
# ]

# cfg = hydra_read(overrides)
# OmegaConf.set_struct(cfg, False)  # This allows getattr and hasattr methods to function correctly
# cfg.data.load_m2f_masks = True   # load input masks
# cfg.data.m2f_preds_dirname = MASK_NAME
# cfg.data.n_views = cfg.models[model_name].backbone.transformer.n_views
# print(cfg.data.n_views)

# # Dataset instantiation
# start = time.time()
# dataset = ScannetDatasetMM(cfg.data)
# print(f"Dataset Creation Time = {time.time() - start:0.1f} sec.")


# # Transformer
# # ViT_masks 6 views small
# checkpoint_dir = '/home/fsun/DeepViewAgg/outputs/ViT_masks_3rd_run' # 3rd run


# evaluator = Evaluator(cfg, checkpoint_dir, early_break=True)
# evaluator.eval_all_metrics(stage_name='val')

In [36]:
### Note: set over to Python file with argparse from script file
MASK_NAME = 'ViT_masks'


# # # DeepSet - DeepSetAttention (check model/config correctness)
# # checkpoint_dir = '/home/fsun/DeepViewAgg/outputs/2023-02-11/10-52-09'   # ViT_masks
# checkpoint_dir = '/home/fsun/DeepViewAgg/outputs/2023-02-11/10-54-19'   # m2f_masks
# dataset_config = 'segmentation/multimodal/Feng/scannet-neucon-smallres-m2f-allviews.yaml'   
# models_config = 'segmentation/multimodal/Feng/view_selection_experiment'    # model family
# model_name = 'DeepSetAttention'                       # specific model



# Transformer - MVFusion_orig
checkpoint_dir = '/home/fsun/DeepViewAgg/outputs/MVFusion_orig'   # ViT_masks
# checkpoint_dir = '/home/fsun/DeepViewAgg_31-10-22/DeepViewAgg/outputs/2023-02-11/22-17-12'   # m2f_masks
dataset_config = 'segmentation/multimodal/Feng/scannet-neucon-smallres-m2f-allviews.yaml'   
models_config = 'segmentation/multimodal/Feng/mvfusion_orig'    # model family
model_name = 'MVFusion_orig'                       # specific model

overrides = [
    'task=segmentation',
    f'data={dataset_config}',
    f'data.dataroot=/scratch-shared/fsun/dvata',
    f'models={models_config}',
    f'model_name={model_name}',
]

cfg = hydra_read(overrides)
OmegaConf.set_struct(cfg, False)  # This allows getattr and hasattr methods to function correctly
cfg.data.load_m2f_masks = True   # load input masks
cfg.data.m2f_preds_dirname = MASK_NAME
cfg.data.n_views = cfg.models[model_name].backbone.transformer.n_views
print(cfg.data.n_views)

# # Dataset instantiation
# start = time.time()
# dataset = ScannetDatasetMM(cfg.data)
# print(f"Dataset Creation Time = {time.time() - start:0.1f} sec.")





9


In [37]:
# ViT transformer
evaluator = Evaluator(cfg, checkpoint_dir, early_break=True)
evaluator.eval_all_metrics(stage_name='val')

self.no_3d ==  True
dataset_class:  scannet.ScannetDatasetMM
dataset_paths:  ['scannet', 'ScannetDatasetMM']
module:  scannet
class_name:  ScannetDatasetMM
dataset_module:  torch_points3d.datasets.segmentation.multimodal.scannet
Load predicted 2D semantic segmentation labels from directory  ViT_masks
initialize train dataset
initialize val dataset
task:  segmentation.multimodal
tested_model_name:  MVFusion_orig
loading gt mask from :  label-filt-scannet20
class_name:  MVFusion_model_orig
model_module:  torch_points3d.models.segmentation.multimodal.Feng.mvfusion_orig
name, cls of chosen model_cls:  MVFusion_model_orig <class 'torch_points3d.models.segmentation.multimodal.Feng.mvfusion_orig.MVFusion_model_orig'>
opt:   {'class': 'Feng.mvfusion_orig.MVFusion_model_orig', 'down_conv': {'image': {'down_conv': {'module_name': 'ADE20KResNet18PPM', 'frozen': False}, 'atomic_pooling': {'module_name': 'BimodalCSRPool', 'mode': 'max'}, 'view_pooling': {'module_name': 'GroupBimodalCSRPool', 'in_ma

  0%|          | 0/312 [00:00<?, ?it/s]

loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt

  0%|          | 0/312 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

...

Adding instance labels to mm_data
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  ViT_masks_refined/MVFusion_orig
3D seen points metrics
{'train_acc': 93.79741124051775, 'train_macc': 91.86903012824285, 'train_miou': 65.15728800349463}
2D cross-view consistency (CC) for input masks
tensor([ 82.9425,  89.0253,  52.3310, 100.0000,  67.2810, 100.0000,  79.4555,
         71.6746,  63.6450, 100.0000, 100.0000,  60.6206, 100.0000, 100.0000,
         70.7917, 100.0000, 100.0000,  72.8202, 100.0000,  59.6130])
Mean:  tensor(83.5100)
2D cross-view consistency (CC) for refined masks
tensor([ 88.7820,  97.2401,  67.7683, 100.0000,  75.9749, 100.0000,  84.5166,
         73.0599,  65.9982, 100.0000, 100.0000,  80.3077, 100.0000, 100.0000,
         77.3859, 100.0000, 100.0000,  83.7078, 100.0000,  79.2469])
Mean:  tensor(88.6994)


2023-02-14 23:29:45,606 - mmflow - INFO - Freeze the parameters in FlowNetCSS
2023-02-14 23:29:46,571 - mmflow - INFO - Freeze the parameters in FlowNetS


load checkpoint from local path: /home/fsun/DeepViewAgg/flow/pretrained/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth


  0%|          | 0/312 [00:00<?, ?it/s]

Temporal Consistency Scores measured over all validation scenes, using all image views
Input 2D {'val_acc': 94.00580451862697, 'val_macc': 70.7263087773185, 'val_miou': 61.04101498307255}
{0: '91.14', 1: '92.82', 2: '89.00', 3: '0.00', 4: '85.65', 5: '56.00', 6: '87.46', 7: '85.00', 8: '92.20', 9: '0.00', 10: '32.13', 11: '88.95', 12: '0.00', 13: '0.34', 14: '89.57', 15: '0.00', 16: '0.00', 17: '79.15', 18: '0.00', 19: '68.28'}
Refined 2D  {'val_acc': 92.65496186838705, 'val_macc': 82.60245826102171, 'val_miou': 74.65474130572322}
{0: '90.49', 1: '91.15', 2: '89.95', 3: '0.00', 4: '77.91', 5: '42.24', 6: '80.47', 7: '87.76', 8: '90.92', 9: '0.00', 10: '14.87', 11: '88.49', 12: '0.00', 13: '42.90', 14: '90.35', 15: '0.00', 16: '0.00', 17: '79.08', 18: '0.00', 19: '78.57'}


In [35]:
# m2f transformer
evaluator = Evaluator(cfg, checkpoint_dir, early_break=True)
evaluator.eval_all_metrics(stage_name='val')

self.no_3d ==  True
dataset_class:  scannet.ScannetDatasetMM
dataset_paths:  ['scannet', 'ScannetDatasetMM']
module:  scannet
class_name:  ScannetDatasetMM
dataset_module:  torch_points3d.datasets.segmentation.multimodal.scannet
Load predicted 2D semantic segmentation labels from directory  m2f_masks
initialize train dataset
initialize val dataset
task:  segmentation.multimodal
tested_model_name:  MVFusion_orig
loading gt mask from :  label-filt-scannet20
class_name:  MVFusion_model_orig
model_module:  torch_points3d.models.segmentation.multimodal.Feng.mvfusion_orig
name, cls of chosen model_cls:  MVFusion_model_orig <class 'torch_points3d.models.segmentation.multimodal.Feng.mvfusion_orig.MVFusion_model_orig'>
opt:   {'class': 'Feng.mvfusion_orig.MVFusion_model_orig', 'down_conv': {'image': {'down_conv': {'module_name': 'ADE20KResNet18PPM', 'frozen': False}, 'atomic_pooling': {'module_name': 'BimodalCSRPool', 'mode': 'max'}, 'view_pooling': {'module_name': 'GroupBimodalCSRPool', 'in_ma

  0%|          | 0/312 [00:00<?, ?it/s]

loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt

  0%|          | 0/312 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

...

3D seen points metrics
{'train_acc': 91.9195483415667, 'train_macc': 86.95368068308608, 'train_miou': 53.50899304615966}
2D cross-view consistency (CC) for input masks
tensor([ 80.1203,  90.3580,  49.8142, 100.0000,  68.3887, 100.0000,  75.4359,
         61.5904,  59.4355, 100.0000, 100.0000,  57.9461, 100.0000, 100.0000,
         51.3328, 100.0000, 100.0000,  57.6782, 100.0000,  65.3593])
Mean:  tensor(80.8730)
2D cross-view consistency (CC) for refined masks
tensor([ 89.2314,  97.3623,  52.9754, 100.0000,  76.2357, 100.0000,  88.1412,
         62.4035,  69.0131, 100.0000, 100.0000,  83.2915, 100.0000, 100.0000,
         70.0636, 100.0000, 100.0000,  64.5703, 100.0000,  81.1318])
Mean:  tensor(86.7210)


2023-02-14 23:26:09,009 - mmflow - INFO - Freeze the parameters in FlowNetS


load checkpoint from local path: /home/fsun/DeepViewAgg/flow/pretrained/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth


  0%|          | 0/312 [00:00<?, ?it/s]

Temporal Consistency Scores measured over all validation scenes, using all image views
Input 2D {'val_acc': 92.4524062713065, 'val_macc': 75.93714775075779, 'val_miou': 67.76782871379214}
{0: '89.30', 1: '91.80', 2: '86.59', 3: '0.00', 4: '83.16', 5: '50.38', 6: '85.73', 7: '72.31', 8: '74.14', 9: '0.00', 10: '54.88', 11: '84.36', 12: '5.26', 13: '0.00', 14: '85.25', 15: '0.00', 16: '0.00', 17: '77.48', 18: '0.00', 19: '75.87'}
Refined 2D  {'val_acc': 92.27986240203396, 'val_macc': 77.82222088077033, 'val_miou': 69.0991861994545}
{0: '90.02', 1: '90.83', 2: '88.75', 3: '0.00', 4: '78.09', 5: '33.86', 6: '80.54', 7: '81.81', 8: '84.42', 9: '0.00', 10: '14.83', 11: '87.55', 12: '42.10', 13: '12.79', 14: '90.31', 15: '0.00', 16: '0.00', 17: '79.50', 18: '0.00', 19: '81.11'}


In [23]:
# ViT deepset
evaluator = Evaluator(cfg, checkpoint_dir, early_break=True)
evaluator.eval_all_metrics(stage_name='val')

self.no_3d ==  True
dataset_class:  scannet.ScannetDatasetMM
dataset_paths:  ['scannet', 'ScannetDatasetMM']
module:  scannet
class_name:  ScannetDatasetMM
dataset_module:  torch_points3d.datasets.segmentation.multimodal.scannet
Load predicted 2D semantic segmentation labels from directory  ViT_masks
initialize train dataset
initialize val dataset
task:  segmentation.multimodal
tested_model_name:  DeepSetAttention
loading gt mask from :  label-filt-scannet20
class_name:  ViewSelectionExp_model
model_module:  torch_points3d.models.segmentation.multimodal.Feng.view_selection_experiment
name, cls of chosen model_cls:  ViewSelectionExp_model <class 'torch_points3d.models.segmentation.multimodal.Feng.view_selection_experiment.ViewSelectionExp_model'>
opt:   {'class': 'Feng.view_selection_experiment.ViewSelectionExp_model', 'down_conv': {'image': {'down_conv': {'module_name': 'ADE20KResNet18PPM', 'frozen': False}, 'atomic_pooling': {'module_name': 'BimodalCSRPool', 'mode': 'max'}, 'view_pool

  0%|          | 0/312 [00:00<?, ?it/s]

loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20


KeyboardInterrupt: 

In [33]:
# m2f deepset
evaluator = Evaluator(cfg, checkpoint_dir, early_break=True)
evaluator.eval_all_metrics(stage_name='val')

self.no_3d ==  True
dataset_class:  scannet.ScannetDatasetMM
dataset_paths:  ['scannet', 'ScannetDatasetMM']
module:  scannet
class_name:  ScannetDatasetMM
dataset_module:  torch_points3d.datasets.segmentation.multimodal.scannet
Load predicted 2D semantic segmentation labels from directory  ViT_masks
initialize train dataset
initialize val dataset
task:  segmentation.multimodal
tested_model_name:  DeepSetAttention
loading gt mask from :  label-filt-scannet20
class_name:  ViewSelectionExp_model
model_module:  torch_points3d.models.segmentation.multimodal.Feng.view_selection_experiment
name, cls of chosen model_cls:  ViewSelectionExp_model <class 'torch_points3d.models.segmentation.multimodal.Feng.view_selection_experiment.ViewSelectionExp_model'>
opt:   {'class': 'Feng.view_selection_experiment.ViewSelectionExp_model', 'down_conv': {'image': {'down_conv': {'module_name': 'ADE20KResNet18PPM', 'frozen': False}, 'atomic_pooling': {'module_name': 'BimodalCSRPool', 'mode': 'max'}, 'view_pool

  0%|          | 0/312 [00:00<?, ?it/s]

loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt

  0%|          | 0/312 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

...

3D seen points metrics
{'train_acc': 93.59731600389067, 'train_macc': 90.48028163474247, 'train_miou': 64.1452106000754}
2D cross-view consistency (CC) for input masks
tensor([ 82.6041,  89.0822,  52.6096, 100.0000,  67.2373, 100.0000,  79.4592,
         70.8621,  63.1024, 100.0000, 100.0000,  60.6106, 100.0000, 100.0000,
         70.5481, 100.0000, 100.0000,  72.4597, 100.0000,  59.0373])
Mean:  tensor(83.3806)
2D cross-view consistency (CC) for refined masks
tensor([ 88.3302,  96.7109,  75.5673, 100.0000,  76.9257, 100.0000,  84.9107,
         65.0874,  55.7636, 100.0000, 100.0000,  81.8995, 100.0000, 100.0000,
         84.1299, 100.0000, 100.0000,  68.5309, 100.0000,  85.4362])
Mean:  tensor(88.1646)


2023-02-14 23:09:53,860 - mmflow - INFO - Freeze the parameters in FlowNetS


load checkpoint from local path: /home/fsun/DeepViewAgg/flow/pretrained/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth


  0%|          | 0/312 [00:00<?, ?it/s]

Temporal Consistency Scores measured over all validation scenes, using all image views
Input 2D {'val_acc': 94.00580451862697, 'val_macc': 70.7263087773185, 'val_miou': 61.04101498307255}
{0: '91.14', 1: '92.82', 2: '89.00', 3: '0.00', 4: '85.65', 5: '56.00', 6: '87.46', 7: '85.00', 8: '92.20', 9: '0.00', 10: '32.13', 11: '88.95', 12: '0.00', 13: '0.34', 14: '89.57', 15: '0.00', 16: '0.00', 17: '79.15', 18: '0.00', 19: '68.28'}
Refined 2D  {'val_acc': 93.01336133557814, 'val_macc': 79.17292835731728, 'val_miou': 72.31756857830301}
{0: '90.62', 1: '91.14', 2: '91.02', 3: '0.00', 4: '79.14', 5: '44.68', 6: '81.99', 7: '87.50', 8: '90.12', 9: '0.00', 10: '14.92', 11: '88.37', 12: '0.00', 13: '0.00', 14: '90.68', 15: '0.00', 16: '0.00', 17: '82.26', 18: '0.00', 19: '79.99'}


In [20]:
# import shutil

# source_dir = "/lisa_migration/project/fsun/data/scannet/scans"

# target_dir = "/scratch-shared/fsun/data/scannet/scans"

# scan_names = np.loadtxt("/home/fsun/data/scannet/splits/scannetv2_val.txt", dtype='str')

# for scan_name in scan_names:
#     color_dir = osp.join(source_dir, scan_name, "color_resized")
    
#     new_color_dir = osp.join(target_dir, scan_name, "color_resized")
    
#     ref_dir = osp.join(target_dir, scan_name, "m2f_masks")
    
#     if len(os.listdir(new_color_dir)) < 10000:
#         print(scan_name)
#         for im in os.listdir(ref_dir):

#     #         print(f"mkdir {mask_dir}")

# #             print(f"cp {osp.join(color_dir, im)} {osp.join(new_color_dir, im)}")
#             shutil.copy(osp.join(color_dir, im), osp.join(new_color_dir, im))
    
#             break
#         break
#     break



In [ ]:
# scans_dir = ""
# scan_names = list(dataset.val_dataset.MAPPING_IDX_TO_SCAN_VAL_NAMES.values())
# input_mask_name = 'ViT_masks'
# model_name = "Deepset_3D"
# for scan_name in scan_names:
#     old_dir = osp.join(scans_dir, scan_name, f"{model_name}_{input_mask_name}_refined")
    
# #     temp_dir = osp.join(scans_dir, scan_name, f"{model_name}_{input_mask_name}_refined_temp")

#     mask_dir = osp.join(scans_dir, scan_name, f"{input_mask_name}_refined")
    
#     output_dir = osp.join(mask_dir, f"{model_name}")
    
#     print(f"mkdir {mask_dir}")
    
#     print(f"mv {old_dir} {output_dir}")
    


In [ ]:
# scans_dir = ""
# scan_names = list(dataset.val_dataset.MAPPING_IDX_TO_SCAN_VAL_NAMES.values())
# input_mask_name = 'm2f_masks'
# temp_name = "MVFusion_3D_small_6views"
# for scan_name in scan_names:
#     old_dir = osp.join(scans_dir, scan_name, f"{input_mask_name}_refined")
    
#     temp_dir = osp.join(scans_dir, scan_name, f"{temp_name}")

#     mask_dir = osp.join(scans_dir, scan_name, f"{input_mask_name}_refined")
#     print(f"mv {old_dir} {temp_dir}")
    
#     print(f"mkdir {mask_dir}")
    
#     print(f"mv {temp_dir} {mask_dir}")
    

In [ ]:
# scans_dir = "/scratch-shared/fsun/data/scannet/scans"

# source_dir = "/home/fsun/data/scannet/scans"

# scan_names = list(dataset.val_dataset.MAPPING_IDX_TO_SCAN_VAL_NAMES.values())
# for s in scan_names:
    
#     print(f"cp -r {osp.join(source_dir, s, 'ViT_masks')} {osp.join(scans_dir, s, 'ViT_masks')}")
    

### Cross-view entropy

- 1. gather all points of an instance

For now, an easy way to measure entropy is to:
- 2. aggregate all predictions from active views of this instance,
- 3a. group instances per semantic class
- 3b. loop over all validation scenes
- 4. measure the entropy per semantic class 

Better method to capture entropy is:
- Measure entropy score for each instance (because aggregating over all scenes puts a focus on label correctness instead of surface consistency)
- Then aggregate entropy scores between all scenes

In [97]:
# input_entropy_tracker = ScannetSegmentationTracker(dataset=dataset, stage='train', wandb_log=False, use_tensorboard=False, ignore_label=IGNORE_LABEL)
# output_entropy_tracker = ScannetSegmentationTracker(dataset=dataset, stage='val', wandb_log=False, use_tensorboard=False, ignore_label=IGNORE_LABEL)
# gt_entropy_tracker = ScannetSegmentationTracker(dataset=dataset, stage='val', wandb_log=False, use_tensorboard=False, ignore_label=IGNORE_LABEL)


# idx = 0
# mapping_idx_to_scan_names = getattr(dataset.val_dataset, "MAPPING_IDX_TO_SCAN_{}_NAMES".format(dataset.val_dataset.split.upper()))
# scan_name = mapping_idx_to_scan_names[idx]

In [98]:
# instance_labels = get_instance_labels(dataset.val_dataset, scan_name)
# mm_data = dataset.val_dataset[idx]

In [99]:
# mm_data.data['instance_labels'] = instance_labels
# mm_data


In [11]:
def get_normalized_entropy(labels):
    counts = torch.unique(labels, return_counts=True)[1]
    
    pk = counts / counts.sum()
    len_pk = torch.tensor(len(pk))
    if len_pk == 1:
        normalized_entropy = 0.
    else:
        normalized_entropy = -sum(pk * torch.log2(pk)) / torch.log2(len_pk)
    return normalized_entropy
    
tracker = ScannetSegmentationTracker(dataset=dataset, stage='train', wandb_log=False, use_tensorboard=False, ignore_label=IGNORE_LABEL)
tracker.reset(stage='train')

instance_count = torch.zeros(2, 20)
cum_n_entropy = torch.zeros(2, 20)

for mm_data in dataset._val_loader:
    print(mm_data.id_scan)
    
    with torch.no_grad():
        model.set_input(mm_data, model.device)
        model(mm_data)

    mm_data.data.pred = model.output.detach().cpu().argmax(1)
    
    mm_data = get_seen_points(mm_data)
    
    tracker.track(model=None, pred_labels=mm_data.data.pred, gt_labels=mm_data.data.y)


    for instance_id in mm_data.data['instance_labels'].unique():

        if instance_id == 0:
            continue

        instance_mask = mm_data.data['instance_labels'] == instance_id
        instance = mm_data[instance_mask]
        instance_class = instance.y[0]
        
        # Skip invalid semantic class
        if instance_class == -1:
            continue

        # Track all per-point predictions of active views for current instance. Some points have more 
        # predictions than others.
        input_preds = instance.modalities['image'][0].get_mapped_m2f_features().squeeze()
        output_preds = instance.modalities['image'][0].get_mapped_gt_labels().squeeze() + 1   # +1 label offset

        input_n_entropy = get_normalized_entropy(input_preds)
        output_n_entropy = get_normalized_entropy(output_preds)
                
        cum_n_entropy[0, instance_class] += input_n_entropy
        cum_n_entropy[1, instance_class] += output_n_entropy
        instance_count[:, instance_class] += 1
        
                
        

    #     # GT views contain invalid label, so mask those points out
    #     output_preds_valid_mask = output_preds != -1

#         gt_label = instance.y.unique()    
#         gt_labels = gt_label.repeat(len(input_preds))

#         input_entropy_tracker.track(model=None, pred_labels=input_preds, gt_labels=gt_labels)
#         output_entropy_tracker.track(model=None, pred_labels=output_preds, 
#                                      gt_labels=gt_labels)
        


    #     input_mask_labels, input_mask_counts = instance.modalities['image'][0].get_mapped_m2f_features().unique(return_counts=True)
    #     print(input_mask_labels, input_mask_counts)

print("3D seen points metrics")
print(tracker.get_metrics())
print(cum_n_entropy)
print(instance_count)

print("Final normalized entropy matrix over all instances")
print(cum_n_entropy / (instance_count + 1e-8))

""" ViT Masks refined
3D seen points metrics
{'train_acc': 92.66021202577072, 'train_macc': 83.70569221362946, 'train_miou': 75.68768085547666}
tensor([[312.2038,  26.8408,  94.0868,  14.3544, 503.6727,  21.2325, 122.7478,
         150.3396,  97.9678,  15.0548, 109.4467,  19.5293,  34.5468,  19.8944,
          14.7285,   8.3424,  18.4457,  42.0031,   4.7867, 179.6838],
        [229.2438,   9.2486,  56.4155,   7.0754, 120.2393,   7.9409,  44.4460,
         102.9044,  87.2189,   6.8330,  76.6685,  13.7709,  12.5995,  12.4757,
           5.0266,   3.5362,   5.3676,  36.9687,   2.2857,  89.8911]])
tensor([[1781.,  319.,  372.,   81., 1367.,   96.,  350.,  465.,  282.,   77.,
          222.,   52.,  127.,   67.,   57.,   28.,   58.,   98.,   31.,  528.],
        [1781.,  319.,  372.,   81., 1367.,   96.,  350.,  465.,  282.,   77.,
          222.,   52.,  127.,   67.,   57.,   28.,   58.,   98.,   31.,  528.]])
Final normalized entropy matrix over all instances
tensor([[0.1753, 0.0841, 0.2529, 0.1772, 0.3685, 0.2212, 0.3507, 0.3233, 0.3474,
         0.1955, 0.4930, 0.3756, 0.2720, 0.2969, 0.2584, 0.2979, 0.3180, 0.4286,
         0.1544, 0.3403],
        [0.1287, 0.0290, 0.1517, 0.0874, 0.0880, 0.0827, 0.1270, 0.2213, 0.3093,
         0.0887, 0.3454, 0.2648, 0.0992, 0.1862, 0.0882, 0.1263, 0.0925, 0.3772,
         0.0737, 0.1702]])
         
         
M2F_masks
3D seen points metrics
{'train_acc': 92.00312846686491, 'train_macc': 83.197711066367, 'train_miou': 74.68883256172798}
tensor([[344.9477,  29.3845, 104.1460,  15.6663, 452.5604,  25.0990, 128.4360,
         163.6485, 112.5904,  15.9026, 122.6382,  15.3401,  34.5984,  22.6733,
          16.1505,   7.8702,  15.1792,  42.9855,   5.7067, 185.2332],
        [188.5569,  12.3578,  44.5499,   7.4627, 406.3045,   9.0966,  76.6618,
          99.0183,  89.4181,  12.0120,  71.5745,  10.2352,  20.3377,   9.4022,
           8.2375,   5.6555,  11.5642,  25.0575,   1.6539, 108.4080]])
tensor([[1783.,  319.,  372.,   81., 1366.,   96.,  350.,  466.,  282.,   77.,
          222.,   52.,  127.,   67.,   57.,   28.,   58.,   98.,   31.,  530.],
        [1783.,  319.,  372.,   81., 1366.,   96.,  350.,  466.,  282.,   77.,
          222.,   52.,  127.,   67.,   57.,   28.,   58.,   98.,   31.,  530.]])
Final normalized entropy matrix over all instances
tensor([[0.1935, 0.0921, 0.2800, 0.1934, 0.3313, 0.2614, 0.3670, 0.3512, 0.3993,
         0.2065, 0.5524, 0.2950, 0.2724, 0.3384, 0.2833, 0.2811, 0.2617, 0.4386,
         0.1841, 0.3495],
        [0.1058, 0.0387, 0.1198, 0.0921, 0.2974, 0.0948, 0.2190, 0.2125, 0.3171,
         0.1560, 0.3224, 0.1968, 0.1601, 0.1403, 0.1445, 0.2020, 0.1994, 0.2557,
         0.0534, 0.2045]])
         
"""

Adding instance labels to mm_dataAdding instance labels to mm_data
filename: Adding instance labels to mm_data
Adding instance labels to mm_data 
filename: Adding instance labels to mm_data/scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename: Adding instance labels to mm_dataAdding instance labels to mm_dataAdding instance labels to mm_dataAdding instance labels to mm_data
filename:  Adding instance labels to mm_data

filename: filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
/scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv

filename: filename: filename:   /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0050_01/scene0050_01_vh_clean_2.ply/scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0

filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0193_00/scene0193_00_vh_clean_2.ply
tensor([26])
63
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0193_01/scene0193_01_vh_clean_2.ply
tensor([27])
36
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0196_00/scene0196_00_vh_clean_2.ply
tensor([28])
180
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0203_00/scene0203_00_vh_clean_2.ply
tensor([29])
Adding instance labels to mm_data
filenam

filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0356_01/scene0356_01_vh_clean_2.ply
tensor([85])
99
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0356_02/scene0356_02_vh_clean_2.ply
tensor([86])
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0357_00/scene0357_00_vh_clean_2.ply
tensor([87])
72
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0357_01/scene0357_01_vh_clean_2.ply
tensor([88])
81
81
Adding instance labels to mm_data
filen

filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0552_00/scene0552_00_vh_clean_2.ply
tensor([144])
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0552_01/scene0552_01_vh_clean_2.ply
tensor([145])
99
54
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0553_00/scene0553_00_vh_clean_2.ply
tensor([146])
90
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallre

Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0629_02/scene0629_02_vh_clean_2.ply
tensor([203])
225
126
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0633_00/scene0633_00_vh_clean_2.ply
tensor([204])
99
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0633_01/scene0633_01_vh_clean_2.ply
tensor([205])
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shar

filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0696_02/scene0696_02_vh_clean_2.ply
tensor([261])
279
180
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0697_00/scene0697_00_vh_clean_2.ply
tensor([262])
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0697_01/scene0697_01_vh_clean_2.ply
tensor([263])
162
Adding instance labels to mm_data
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/metadata/scannetv2-labels.combined.tsv
filename:  /scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/raw/scans/scene0697_02/scene0697_02_vh_clean_2.ply
tensor([264])
153
Adding instance labels to mm_da

" ViT Masks refined\n3D seen points metrics\n{'train_acc': 92.66021202577072, 'train_macc': 83.70569221362946, 'train_miou': 75.68768085547666}\ntensor([[312.2038,  26.8408,  94.0868,  14.3544, 503.6727,  21.2325, 122.7478,\n         150.3396,  97.9678,  15.0548, 109.4467,  19.5293,  34.5468,  19.8944,\n          14.7285,   8.3424,  18.4457,  42.0031,   4.7867, 179.6838],\n        [229.2438,   9.2486,  56.4155,   7.0754, 120.2393,   7.9409,  44.4460,\n         102.9044,  87.2189,   6.8330,  76.6685,  13.7709,  12.5995,  12.4757,\n           5.0266,   3.5362,   5.3676,  36.9687,   2.2857,  89.8911]])\ntensor([[1781.,  319.,  372.,   81., 1367.,   96.,  350.,  465.,  282.,   77.,\n          222.,   52.,  127.,   67.,   57.,   28.,   58.,   98.,   31.,  528.],\n        [1781.,  319.,  372.,   81., 1367.,   96.,  350.,  465.,  282.,   77.,\n          222.,   52.,  127.,   67.,   57.,   28.,   58.,   98.,   31.,  528.]])\nFinal normalized entropy matrix over all instances\ntensor([[0.1753, 

In [12]:
print(cum_n_entropy / (instance_count + 1e-8) * 100)


tensor([[19.3465,  9.2115, 27.9962, 19.3411, 33.1303, 26.1448, 36.6960, 35.1177,
         39.9257, 20.6528, 55.2424, 29.5002, 27.2429, 33.8408, 28.3342, 28.1079,
         26.1711, 43.8628, 18.4088, 34.9497],
        [10.5753,  3.8739, 11.9758,  9.2132, 29.7441,  9.4756, 21.9034, 21.2486,
         31.7085, 15.6000, 32.2408, 19.6831, 16.0139, 14.0331, 14.4517, 20.1982,
         19.9383, 25.5689,  5.3351, 20.4543]])


# Evaluate 2D input mask mIoU

In [24]:
scan_names = np.loadtxt("/home/fsun/data/scannet/splits/scannetv2_val.txt", dtype=str)
scans_dir = "/scratch-shared/fsun/data/scannet/scans"

input_mask_name = 'ViT_masks'

tracker = dataset.get_tracker(False, False)
    
tracker.reset(stage="train")

for scan_name in Ctq(scan_names):

    gt_dir = osp.join(scans_dir, scan_name, 'label-filt-scannet20')
    mask_dir = osp.join(scans_dir, scan_name, f"{input_mask_name}")

    print(len(os.listdir(mask_dir)))
    for im in os.listdir(mask_dir):
        
        gt = Image.open(osp.join(gt_dir, im))
        mask = Image.open(osp.join(mask_dir, im))
        gt = gt.resize((640, 480), 0)

        gt = np.asarray(gt).astype(float) - 1
        mask = np.asarray(mask) - 1

        tracker.track(pred_labels=mask, gt_labels=gt, model=None)

print("2D evaluation of input masks")
print(tracker.get_metrics())
print(tracker._miou_per_class)  

  0%|          | 0/312 [00:00<?, ?it/s]

189
171
171
225
36
108
225
144
108
36
54
99
297
324
261
189
171
117
99
171
252
144
144
135
162
117
369
288
387
180
171
153
117
126
117
126
72
108
180
333
252
162
261
207
63
117
63
54
135
117
135
171
207
333
369
360
108
90
117
63
36
126
54
72
63
45
90
180
198
234
270
99
144
144
126
117
144
117
81
126
189
324
216
153
162
171
27
36
180
162
180
90
54
108
234
126
126
81
81
63
117
126
207
135
189
45
45
342
378
99
63
144
81
54
126
306
108
45
81
126
126
126
162
153
63
81
54
99
54
252
225
126
162
99
63
108
126
54
54
54
126
108
63
72
45
54
126
108
198
99
171
153
117
135
279
180
126
63
162
225
495
558
333
27
234
171
162
180
63
189
171
153
90
63
108
207
126
234
342
504
189
126
117
99
72
315
225
243
342
171
72
54
27
153
189
171
153
90
81
99
45
234
270
234
54
54
54
189
189
216
180
180
198
198
162
297
162
81
216
135
135
126
108
216
225
90
108
99
360
36
45
171
144
324
243
162
153
108
207
180
234
36
342
405
144
225
36
54
45
27
198
117
81
81
324
252
216
99
54
270
342
279
306
189
189
54
45
72
72
72
333
3

In [12]:
scan_names = np.loadtxt("/home/fsun/data/scannet/splits/scannetv2_val.txt", dtype=str)
scans_dir = "/scratch-shared/fsun/data/scannet/scans"

input_mask_name = 'm2f_masks'

tracker = dataset.get_tracker(False, False)
    
tracker.reset(stage="train")

for scan_name in Ctq(scan_names):

    gt_dir = osp.join(scans_dir, scan_name, 'label-filt-scannet20')
    mask_dir = osp.join(scans_dir, scan_name, f"{input_mask_name}")

    print(len(os.listdir(mask_dir)))
    for im in os.listdir(mask_dir):
        
        gt = Image.open(osp.join(gt_dir, im))
        mask = Image.open(osp.join(mask_dir, im))
        gt = gt.resize((640, 480), 0)

        gt = np.asarray(gt).astype(float) - 1
        mask = np.asarray(mask) - 1

        tracker.track(pred_labels=mask, gt_labels=gt, model=None)

print("2D evaluation of input masks")
print(tracker.get_metrics())
print(tracker._miou_per_class)  

  0%|          | 0/312 [00:00<?, ?it/s]

189
171
171
225
36
108
225
144
108
36
54
99
297
324
261
189
171
117
99
171
252
144
144
135
162
117
369
288
387
180
171
153
117
126
117
126
72
108
180
333
252
162
261
207
63
117
63
54
135
117
135
171
207
333
369
360
108
90
117
63
36
126
54
72
63
45
90
180
198
234
270
99
144
144
126
117
144
117
81
126
189
324
216
153
162
171
27
36
180
162
180
90
54
108
234
126
126
81
81
63
117
126
207
135
189
45
45
342
378
99
63
144
81
54
126
306
108
45
81
126
126
126
162
153
63
81
54
99
54
252
225
126
162
99
63
108
126
54
54
54
126
108
63
72
45
54
126
108
198
99
171
153
117
135
279
180
126
63
162
225
495
558
333
27
234
171
162
180
63
189
171
153
90
63
108
207
126
234
342
504
189
126
117
99
72
315
225
243
342
171
72
54
27
153
189
171
153
90
81
99
45
234
270
234
54
54
54
189
189
216
180
180
198
198
162
297
162
81
216
135
135
126
108
216
225
90
108
99
360
36
45
171
144
324
243
162
153
108
207
180
234
36
342
405
144
225
36
54
45
27
198
117
81
81
324
252
216
99
54
270
342
279
306
189
189
54
45
72
72
72
333
3

# Temporary evaluation of majority vote + projected baseline masks

In [14]:
import os
import os.path as osp
import copy
import torch
import hydra
import time
import logging
import scipy.ndimage
import numpy as np
from PIL import Image
import open3d as o3d

# Import building function for model and dataset
from torch_points3d.datasets.dataset_factory import instantiate_dataset
from torch_points3d.models.model_factory import instantiate_model

# Import BaseModel / BaseDataset for type checking
from torch_points3d.models.base_model import BaseModel
from torch_points3d.datasets.base_dataset import BaseDataset

# Import from metrics
from torch_points3d.metrics.base_tracker import BaseTracker
from torch_points3d.metrics.colored_tqdm import Coloredtqdm as Ctq
from torch_points3d.metrics.model_checkpoint import ModelCheckpoint

# Utils import
from torch_points3d.utils.colors import COLORS
from torch_points3d.utils.wandb_utils import Wandb
from torch_points3d.utils.config import getattr_recursive
from torch_points3d.visualization import Visualizer
from torch_points3d.core.data_transform.transforms import PointcloudMerge
from torch_points3d.datasets.segmentation.scannet import CLASS_COLORS, CLASS_NAMES, CLASS_LABELS





log = logging.getLogger(__name__)


class Evaluator():
    
    def __init__(self, cfg, checkpoint_dir, early_break=False):
        self.scans_dir = "/scratch-shared/fsun/data/scannet/scans"

#         self._dataset = dataset
        self._cfg = cfg
    
        if self._cfg.model_name in ['MVFusion_orig', 'DeepSetAttention']:
            self.no_3d = True
        else:
            self.no_3d = False
            
        print("self.no_3d == ", self.no_3d)
            
        self.wandb_log = False
        self.tensorboard_log = False
        
        self.early_break = early_break


#         # Create the model
#         print(f"Creating model: {self._cfg.model_name}")
#         model = instantiate_model(self._cfg, self._dataset)

#         # Load the checkpoint and recover the 'best_miou' model weights
#         checkpoint = torch.load(f'{checkpoint_dir}/{self._cfg.model_name}.pt', map_location='cpu')
#         model.load_state_dict_with_same_shape(checkpoint['models']['best_miou'], strict=False)

#         # Prepare the model for training
#         self._model = model.cuda()
#         self._device = self._model.device

#         print(f"Device is {self._device}")
        
        
        # Enable CUDNN BACKEND
        torch.backends.cudnn.enabled = self.enable_cudnn

        if not self.has_training:
            self._cfg.training = self._cfg
            resume = bool(self._cfg.checkpoint_dir)
        else:
            resume = bool(self._cfg.training.checkpoint_dir)

        # Get device
        if self._cfg.training.cuda > -1 and torch.cuda.is_available():
            device = "cuda"
            torch.cuda.set_device(self._cfg.training.cuda)
        else:
            device = "cpu"
        self._device = torch.device(device)
        log.info("DEVICE : {}".format(self._device))

        
        # Start Wandb if public
        if self.wandb_log:
            Wandb.launch(
                self._cfg, self._cfg.training.wandb.public and self.wandb_log)

        # Checkpoint
        self._checkpoint: ModelCheckpoint = ModelCheckpoint(
            checkpoint_dir, self._cfg.model_name,
            "", run_config=self._cfg, resume=resume)
            
        # Recover the merged config from Checkpoint
        self._cfg = self._checkpoint.run_config

        # Update dataset config with number of views for the MVFusion Transformer
        model_config = getattr(self._cfg.models, self._cfg.model_name, None)
        self._cfg.data.n_views = model_config.backbone.transformer.n_views
        
        print("####################### Temporary Solution for baseline + 2D-3D projection: SETTING MODEL_NAME TO : majority_vote")
        print("Number of views for baseline input: ", self._cfg.data.n_views)        
        self._cfg.model_name = 'majority_vote'
        
        # Create model and datasets
        if not self._checkpoint.is_empty:
            self._dataset: BaseDataset = instantiate_dataset(self._cfg.data)
            self._model: BaseModel = self._checkpoint.create_model(
                self._dataset, weight_name="best_miou")
        else:
            log.warning("Checkpoint is empty or model cannot be instantiated from given checkpoint.")

#             self._dataset: BaseDataset = instantiate_dataset(self._cfg.data)
#             self._model: BaseModel = instantiate_model(
#                 copy.deepcopy(self._cfg), self._dataset)
#             self._model.instantiate_optimizers(self._cfg, "cuda" in device)
#             self._model.set_pretrained_weights()
#             if not self._checkpoint.validate(self._dataset.used_properties):
#                 log.warning(
#                     "The model will not be able to be used from pretrained "
#                     "weights without the corresponding dataset. Current "
#                     "properties are {}".format(self._dataset.used_properties))
                
        self._checkpoint.dataset_properties = self._dataset.used_properties

        log.info(self._model)

        self._model.log_optimizers()
        log.info("Model size = %i", sum(param.numel() for param in self._model.parameters() if param.requires_grad))

        # Set dataloaders
        self._dataset.create_dataloaders(
            self._model,
            self._cfg.training.batch_size,
            self._cfg.training.shuffle,
            self._cfg.training.num_workers,
            self.precompute_multi_scale,
            train_only=False,
            val_only=True,
            test_batch_size=1
        )
        
        log.info(self._dataset)

        # Verify attributes in dataset
        self._model.verify_data(self._dataset.train_dataset[0])

        # Choose selection stage
        selection_stage = getattr(self._cfg, "selection_stage", "")
        self._checkpoint.selection_stage = self._dataset.resolve_saving_stage(
            selection_stage)
        
        self.mapping_idx_to_scan_names = getattr(self._dataset.val_dataset, "MAPPING_IDX_TO_SCAN_{}_NAMES".format(self._dataset.val_dataset.split.upper()))

        
    def eval_all_metrics(self, stage_name=""):
        self._is_training = False
        
        self._tracker_refined: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
        self._tracker_refined_seen_points: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)            
                        
        if self._dataset.has_val_loader:
            if not stage_name or stage_name == "val":
                print("Evaluating on validation set")
                self._test_refined(epoch=1, stage_name="val")   
                
        # Upscale predictions containing all points to 0.01 voxel size and save point cloud predictions   
        self._tracker_refined.finalise(full_res=True)
        path_to_submission = save_semantic_prediction_as_txt(
            self._tracker_refined, self._cfg.model_name, self._dataset.val_dataset.m2f_preds_dirname)
        
        # Back-project semantic mesh (from pcd) to 2D images given the maximum number of views per scene, and save.
        # Skips this step if refined images already exist for given model and mask
        self.mesh_to_image(self._cfg, self._dataset, path_to_submission, self.scans_dir, save_output='if_not_exists') 
        
        # Evaluate 2D semantic segmentation
        self._tracker_refined_2d_iou: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
        
        self._evaluate_2d_iou()
        
        # Evaluate 2D cross-view consistency
        # Temporary solution to evaluate both 2D input masks and 2D refined masks for Cross-view Consistency:
        # load refined masks into gt_mask attribute
        self._dataset.val_dataset.gt_dir_name = osp.join(f"{self._dataset.val_dataset.m2f_preds_dirname}_refined", 
                                                         self._cfg.model_name)
        self._evaluate_2d_CC()        
        
        # Evaluate 2D temporal consistency
        self.tracker_TC_baseline: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
        self.tracker_TC_refined: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
        self._evaluate_2d_TC()   
        
    def _evaluate_2d_TC(self):
        from mmflow.apis import init_model, inference_model
        from mmflow.datasets import visualize_flow, write_flow
        import mmcv
        from mmflow.ops import Warp

        # Specify the path to model config and checkpoint file
        config_file = '/home/fsun/DeepViewAgg/flow/configs/flownet2_8x1_sfine_flyingthings3d_subset_384x768.py'
        checkpoint_file = '/home/fsun/DeepViewAgg/flow/pretrained/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth'

        # build the model from a config file and a checkpoint file
        flow_model = init_model(config_file, checkpoint_file, device='cuda:0')

        warp = Warp(mode='nearest',
                    padding_mode='zeros',
                    align_corners=False,
                    use_mask=True).cuda()
        
    
        mask_foldername = self._dataset.val_dataset.m2f_preds_dirname
        refined_mask_foldername = osp.join(f"{mask_foldername}_refined", self._cfg.model_name)
        mask_scans_dir = '/scratch-shared/fsun/data/scannet/scans'
        

        self.tracker_TC_baseline.reset(stage='val')
        self.tracker_TC_refined.reset(stage='val')

        scan_ids = list(self.mapping_idx_to_scan_names.values())

        scans_dir = "/scratch-shared/fsun/data/scannet/scans"

        for scan_id in Ctq(scan_ids):
            scan_dir = osp.join(scans_dir, scan_id)

            mask_scan_dir = osp.join(mask_scans_dir, scan_id)

            mask_dir = osp.join(mask_scan_dir, mask_foldername)
            refined_mask_dir = osp.join(scan_dir, refined_mask_foldername)

            refined_mask_names = os.listdir(refined_mask_dir)
            refined_mask_names = sorted(refined_mask_names, key=lambda i: int(os.path.splitext(os.path.basename(i))[0]))

            color_im_dir = osp.join(scan_dir, 'color_resized')

            # Loop over all pairs of consecutive images
            for i in range(len(refined_mask_names) - 1):
                cur_frame_name, next_frame_name = refined_mask_names[i], refined_mask_names[i+1]
                cur_frame_id, next_frame_id = int(refined_mask_names[i].split(".")[0]), int(refined_mask_names[i+1].split(".")[0])

                # Skip if jump in frames is too big
                if next_frame_id - cur_frame_id > 50:
                    continue

                cur_color_p, next_color_p = osp.join(color_im_dir, cur_frame_name), osp.join(color_im_dir, next_frame_name)
                cur_color_im, next_color_im = Image.open(cur_color_p), Image.open(next_color_p)

                result = inference_model(flow_model, next_color_p, cur_color_p)
                flow_map = torch.tensor(result).permute(2, 0, 1).unsqueeze(0)

                ################# 2D input masks
                seg_im_p1 = osp.join(mask_dir, cur_frame_name)
                seg_im_p2 = osp.join(mask_dir, next_frame_name)
                seg_im1 = np.asarray(Image.open(seg_im_p1)) 
                seg_im2 = np.asarray(Image.open(seg_im_p2)).astype(np.int) - 1   # Adjust labels

                # warping im1 to im2
                seg_im1_semantic = torch.tensor(seg_im1).unsqueeze(0).unsqueeze(0).float()
                seg_im_warped = warp(seg_im1_semantic, flow_map).permute(0, 2, 3, 1)[0].squeeze() - 1    # Adjust labels

                # take im2 as 'pseudo gt' for temporal consistency. Thus, invalidly warped pixels should be ignored
                # by setting the corresponding gt label to the IGNORE_LABEL
                seg_im2[seg_im_warped == -1] = IGNORE_LABEL
                self.tracker_TC_baseline.track(pred_labels=seg_im_warped.long(), gt_labels=seg_im2, model=None)

                ################ 2D refined masks
                seg_im_p1 = osp.join(refined_mask_dir, cur_frame_name)
                seg_im_p2 = osp.join(refined_mask_dir, next_frame_name)
                seg_im1 = np.asarray(Image.open(seg_im_p1))  + 1 # Adjust labels for flow warping (invalid pixels will be set to 0)

                # Resize in case
                seg_im2 = Image.open(seg_im_p2).resize((640, 480), resample=0)
                seg_im2 = np.asarray(seg_im2).astype(np.int) 

                # warping im1 to im2
                seg_im1_semantic = torch.tensor(seg_im1).unsqueeze(0).unsqueeze(0).float()
                seg_im_warped = warp(seg_im1_semantic, flow_map).permute(0, 2, 3, 1)[0].squeeze() - 1 # Adjust labels back to normal

                # take im2 as 'pseudo gt' for temporal consistency. Thus, invalidly warped pixels should be ignored
                # by setting the corresponding gt label to the IGNORE_LABEL
                seg_im2[seg_im_warped == -1] = IGNORE_LABEL
                self.tracker_TC_refined.track(pred_labels=seg_im_warped.long(), gt_labels=seg_im2, model=None)

            if self.early_break:
                break
            
        print("Temporal Consistency Scores measured over all validation scenes, using all image views")
        print("Input 2D", self.tracker_TC_baseline.get_metrics())
        print(self.tracker_TC_baseline._miou_per_class)   
        print("Refined 2D ", self.tracker_TC_refined.get_metrics())
        print(self.tracker_TC_refined._miou_per_class)   

    def _evaluate_2d_CC(self):
        
        temp_tracker: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
                
        # Load instance labels for cross-view consistency evaluation
        self._dataset.val_dataset.load_instance_labels = True
        
#         # Load refined masks in attribute of gt mask for CC evaluation
#         self._dataset.val_dataset.gt_dir_name = osp.join(self._cfg.data.m2f_preds_dirname + '_refined', self._cfg.model_name) 

        
        del self._dataset._val_loader
        
        # Re-create dataloader
        self._dataset.create_dataloaders(
            self._model,
            self._cfg.training.batch_size,
            self._cfg.training.shuffle,
            self._cfg.training.num_workers,
            self.precompute_multi_scale,
            train_only=False,
            val_only=True,
            test_batch_size=1
        )
                
        self.instance_count = torch.zeros(2, 20)
        self.cum_n_entropy = torch.zeros(2, 20)

        self._model.eval()
            
        with Ctq(self._dataset.val_dataloader) as tq_loader:
            for batch in tq_loader:
                
                with torch.no_grad():
#                     self._model.set_input(batch, self._device)
                    
#                     with torch.cuda.amp.autocast(enabled=self._model.is_mixed_precision()):
#                         self._model.forward(epoch=1)

#                     batch.data.pred = self._model.output.detach().cpu().argmax(1)

                    batch = get_seen_points(batch)
    
                    


                    temp_tracker.track(model=None, pred_labels=get_mode_pred(batch), gt_labels=batch.data.y)

                    # Accumulate entropy of seen points
                    self.add_entropy_to_accumulator(batch)
                    
                if self.early_break:
                    break
            
        print("3D seen points metrics")
        print(temp_tracker.get_metrics())
        
        self.cum_n_entropy = self.cum_n_entropy / (self.instance_count + 1e-8)
        self.crossview_consistency = (1 - self.cum_n_entropy) * 100       
        print("2D cross-view consistency (CC) for input masks")
        print(self.crossview_consistency[0])
        print("Mean: ", self.crossview_consistency[0].mean())

        print("2D cross-view consistency (CC) for refined masks")
        print(self.crossview_consistency[1])
        print("Mean: ", self.crossview_consistency[1].mean())

        
                
    def add_entropy_to_accumulator(self, mm_data):
        for instance_id in mm_data.data['instance_labels'].unique():

            if instance_id == 0:
                continue

            instance_mask = mm_data.data['instance_labels'] == instance_id
            instance = mm_data[instance_mask]
            instance_class = instance.y[0]

            # Skip invalid semantic class
            if instance_class == IGNORE_LABEL:
                continue

            # Track all per-point predictions of active views for current instance. Some points have more 
            # predictions than others.
            input_preds = instance.modalities['image'][0].get_mapped_m2f_features().squeeze()
            output_preds = instance.modalities['image'][0].get_mapped_gt_labels().squeeze() + 1   # +1 label offset

            input_n_entropy = get_normalized_entropy(input_preds)
            output_n_entropy = get_normalized_entropy(output_preds)

            self.cum_n_entropy[0, instance_class] += input_n_entropy
            self.cum_n_entropy[1, instance_class] += output_n_entropy
            self.instance_count[:, instance_class] += 1

    def mesh_to_image(self, cfg, dataset, path_to_submission, scans_dir, save_output='if_not_exists'):
        # User input
        output_image_size = (480, 640)
        preprocessed_2d_data_dir = "/scratch-shared/fsun/dvata/scannet-neucon-smallres-m2f/processed/processed_2d_val"

        ########################################################################################################################
        input_mask_name = cfg.data.m2f_preds_dirname
        scan_names = list(dataset.val_dataset.MAPPING_IDX_TO_SCAN_VAL_NAMES.values())

#         path_to_submission = osp.join(path_to_submission, cfg.model_name, input_mask_name)

        for scan_name in Ctq(scan_names):
            # Output folder location
            refined_mask_dir = osp.join(scans_dir, scan_name, f"{input_mask_name}_refined", f"{cfg.model_name}")

            if not osp.exists(refined_mask_dir):
                os.makedirs(refined_mask_dir)

#             # Skip this step if output folder already contains masks
#             if len(os.listdir(refined_mask_dir)) < 10:

            # Load data
            mesh = o3d.io.read_triangle_mesh(f"{scans_dir}/{scan_name}/{scan_name}_vh_clean_2.ply")
            mesh_triangles = np.asarray(mesh.triangles)
            mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)

            # Load predicted class label per vertex
            class_id_faces = np.loadtxt(f"{path_to_submission}/{scan_name}.txt").astype(int)

            # Camera parameters
            intrinsic = np.loadtxt(f"{scans_dir}/{scan_name}/sens/intrinsic/intrinsic_depth.txt")[:3, :3]
            images = torch.load(f"{preprocessed_2d_data_dir}/{scan_name}.pt")

            # Undo axis alignment for extrinsics  
            axis_align_matrix_path = osp.join(scans_dir, scan_name, scan_name + '.txt')
            axis_align_matrix = read_axis_align_matrix(axis_align_matrix_path)
            inv = torch.linalg.inv(axis_align_matrix.T)
            images.extrinsic = inv.T  @ images.extrinsic        

            # Make world-to-camera
            extrinsics = torch.linalg.inv(images.extrinsic).numpy()
            image_names = [osp.splitext(osp.basename(x))[0] for x in images.path]

            # Raycasting
            scene = o3d.t.geometry.RaycastingScene()
            scene.add_triangles(mesh)

            for i in range(len(image_names)):
                image = get_semantic_image_from_camera(dataset=dataset, scene=scene, mesh_triangles=mesh_triangles, intrinsic=intrinsic,
                                                       extrinsic=extrinsics[i], 
                                                       class_id_faces=class_id_faces, im_size=output_image_size)


                # Save refined prediction (backprojected from mesh + interpolated missing pixels)
                image = Image.fromarray(image.astype(np.uint8), 'L')
                im_save_path = osp.join(refined_mask_dir, image_names[i] + '.png')
                image.save(im_save_path)   

#             else:
#                 print("Output directory already exists and contains >10 masks!")

            if self.early_break:
                break
        
    def _evaluate_2d_iou(self):
        input_mask_name = self._cfg.data.m2f_preds_dirname

        scan_names = list(self._dataset.val_dataset.MAPPING_IDX_TO_SCAN_VAL_NAMES.values())
        
        self._tracker_refined_2d_iou.reset(stage="val")

        with Ctq(self._dataset.val_dataloader) as tq_loader:
            for batch in tq_loader:
                scan_name = self.mapping_idx_to_scan_names[batch.id_scan.item()]

                gt_dir = osp.join(self.scans_dir, scan_name, 'label-filt-scannet20')
                mask_dir = osp.join(self.scans_dir, scan_name, f"{input_mask_name}_refined", f"{cfg.model_name}")

                im_names = [osp.basename(x) for x in batch.modalities['image'][0].m2f_pred_mask_path]

                for im in im_names:
                    gt = Image.open(osp.join(gt_dir, im))
                    mask = Image.open(osp.join(mask_dir, im))
                    gt = gt.resize((640, 480), 0)

                    gt = np.asarray(gt).astype(float) - 1
                    mask = np.asarray(mask)

                    self._tracker_refined_2d_iou.track(pred_labels=mask, gt_labels=gt, model=None)
                    
                if self.early_break:
                    break

        print("2D evaluation of refined masks")
        print(self._tracker_refined_2d_iou.get_metrics())
        print(self._tracker_refined_2d_iou._miou_per_class)       
            
    def _test_refined(self, epoch, stage_name: str):

        loaders = [self._dataset.val_dataloader]

        self._model.eval()
            
        for loader in loaders:
            print("Input mask name: ", loader.dataset.m2f_preds_dirname)
            
            stage_name = loader.dataset.name
            self._tracker_refined.reset(stage_name)
            self._tracker_refined_seen_points.reset(stage_name)
            
            with Ctq(loader) as tq_loader:
                for data in tq_loader:
                    with torch.no_grad():
                        if self.no_3d:
                            data = get_seen_points(data)
                            self._model.set_input(data, self._device)
                        else:
                            self._model.set_input(data, self._device)
                            
#                         with torch.cuda.amp.autocast(enabled=self._model.is_mixed_precision()):
#                             self._model.forward(epoch=epoch)


                        self._model.output = torch.nn.functional.one_hot(get_mode_pred(data), num_classes=20)


                        data.data.pred = self._model.output.detach().cpu().argmax(1)

                        # 3D mIoU, all points
                        self._tracker_refined.track(self._model, full_res=True, data=data)

                        if not self.no_3d:
                            # 3D mIoU, seen points
                            data = get_seen_points(data)
                        self._tracker_refined_seen_points.track(pred_labels=data.data.pred, gt_labels=data.data.y, model=None)
        

                    tq_loader.set_postfix(**self._tracker_refined.get_metrics())
                    
                    if self.early_break:
                        break

            print("Evaluated scores for 3D semantic segmentation on all points: ")
            
            print("--- Refined 3D ---")
            print(self._tracker_refined.get_metrics())
            print(self._tracker_refined._miou_per_class)
            
            print("Evaluated scores for 3D semantic segmentation on seen points: ")
    
            print("--- Refined 3D ---")
            print(self._tracker_refined_seen_points.get_metrics())
            print(self._tracker_refined_seen_points._miou_per_class)
            
                            
        
    def eval_baseline(self, stage_name=""):
        self._is_training = False
        
        self._tracker_baseline: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
            
        self._tracker_baseline_2d: BaseTracker = self._dataset.get_tracker(
            self.wandb_log, self.tensorboard_log)
                        
        epoch = 1

        if self._dataset.has_val_loader:
            if not stage_name or stage_name == "val":
                self._test_baseline(epoch, "val")   

    @property
    def has_training(self):
        return getattr(self._cfg, "training", None)
    
    @property
    def enable_cudnn(self):
        if self.has_training:
            return getattr(self._cfg.training, "enable_cudnn", True)
        else:
            return getattr(self._cfg, "enable_cudnn", True)

    @property
    def precompute_multi_scale(self):
        if not self.has_training:
            return self._model.conv_type == "PARTIAL_DENSE" and getattr(self._cfg.training, "precompute_multi_scale", False)
        else:
            return self._model.conv_type == "PARTIAL_DENSE" and getattr(self._cfg, "precompute_multi_scale", False)

In [15]:
### Note: set over to Python file with argparse from script file
MASK_NAME = 'ViT_masks'


# # # DeepSet - DeepSetAttention (check model/config correctness)
# # checkpoint_dir = '/home/fsun/DeepViewAgg/outputs/2023-02-11/10-52-09'   # ViT_masks
# checkpoint_dir = '/home/fsun/DeepViewAgg/outputs/2023-02-11/10-54-19'   # m2f_masks
# dataset_config = 'segmentation/multimodal/Feng/scannet-neucon-smallres-m2f-allviews.yaml'   
# models_config = 'segmentation/multimodal/Feng/view_selection_experiment'    # model family
# model_name = 'DeepSetAttention'                       # specific model



# Transformer - MVFusion_orig
checkpoint_dir = '/home/fsun/DeepViewAgg/outputs/MVFusion_orig'   # ViT_masks
# checkpoint_dir = '/home/fsun/DeepViewAgg_31-10-22/DeepViewAgg/outputs/2023-02-11/22-17-12'   # m2f_masks
dataset_config = 'segmentation/multimodal/Feng/scannet-neucon-smallres-m2f-allviews.yaml'   
models_config = 'segmentation/multimodal/Feng/mvfusion_orig'    # model family
model_name = 'MVFusion_orig'                       # specific model

overrides = [
    'task=segmentation',
    f'data={dataset_config}',
    f'data.dataroot=/scratch-shared/fsun/dvata',
    f'models={models_config}',
    f'model_name={model_name}',
]

cfg = hydra_read(overrides)
OmegaConf.set_struct(cfg, False)  # This allows getattr and hasattr methods to function correctly
cfg.data.load_m2f_masks = True   # load input masks
cfg.data.m2f_preds_dirname = MASK_NAME
cfg.data.n_views = cfg.models[model_name].backbone.transformer.n_views
print(cfg.data.n_views)

# # Dataset instantiation
# start = time.time()
# dataset = ScannetDatasetMM(cfg.data)
# print(f"Dataset Creation Time = {time.time() - start:0.1f} sec.")





9


In [16]:
# ViT transformer
evaluator = Evaluator(cfg, checkpoint_dir, early_break=True)
evaluator.eval_all_metrics(stage_name='val')

self.no_3d ==  True
####################### Temporary Solution for baseline + 2D-3D projection: SETTING MODEL_NAME TO : majority_vote
Number of views for baseline input:  9
dataset_class:  scannet.ScannetDatasetMM
dataset_paths:  ['scannet', 'ScannetDatasetMM']
module:  scannet
class_name:  ScannetDatasetMM
dataset_module:  torch_points3d.datasets.segmentation.multimodal.scannet
Load predicted 2D semantic segmentation labels from directory  ViT_masks
initialize train dataset
initialize val dataset
task:  segmentation.multimodal
tested_model_name:  MVFusion_orig
loading gt mask from :  label-filt-scannet20
class_name:  MVFusion_model_orig
model_module:  torch_points3d.models.segmentation.multimodal.Feng.mvfusion_orig
name, cls of chosen model_cls:  MVFusion_model_orig <class 'torch_points3d.models.segmentation.multimodal.Feng.mvfusion_orig.MVFusion_model_orig'>
opt:   {'class': 'Feng.mvfusion_orig.MVFusion_model_orig', 'down_conv': {'image': {'down_conv': {'module_name': 'ADE20KResNet18

  0%|          | 0/312 [00:00<?, ?it/s]

loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt

  0%|          | 0/312 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

...





Adding instance labels to mm_data
loading gt mask from :  label-filt-scannet20
loading gt mask from :  label-filt-scannet20
loading gt mask from :  ViT_masks_refined/majority_vote
Adding instance labels to mm_data
loading gt mask from :  ViT_masks_refined/majority_vote
Adding instance labels to mm_data
loading gt mask from :  ViT_masks_refined/majority_vote
Adding instance labels to mm_data
loading gt mask from :  ViT_masks_refined/majority_vote
Adding instance labels to mm_data
loading gt mask from :  ViT_masks_refined/majority_vote
loading gt mask from :  ViT_masks_refined/majority_vote
loading gt mask from :  ViT_masks_refined/majority_vote
loading gt mask from :  ViT_masks_refined/majority_vote
loading gt mask from :  ViT_masks_refined/majority_vote
Adding instance labels to mm_data
Adding instance labels to mm_data
loading gt mask from :  ViT_masks_refined/majority_vote
loading gt mask from :  ViT_masks_refined/majority_vote
loading gt mask from :  ViT_masks_refined/majority_v

2023-02-15 14:58:19,048 - mmflow - INFO - Freeze the parameters in FlowNetCSS
2023-02-15 14:58:20,040 - mmflow - INFO - Freeze the parameters in FlowNetS


load checkpoint from local path: /home/fsun/DeepViewAgg/flow/pretrained/flownet2_8x1_sfine_flyingthings3d_subset_384x768.pth


  0%|          | 0/312 [00:00<?, ?it/s]

Temporal Consistency Scores measured over all validation scenes, using all image views
Input 2D {'val_acc': 94.00580451862697, 'val_macc': 70.7263087773185, 'val_miou': 61.04101498307255}
{0: '91.14', 1: '92.82', 2: '89.00', 3: '0.00', 4: '85.65', 5: '56.00', 6: '87.46', 7: '85.00', 8: '92.20', 9: '0.00', 10: '32.13', 11: '88.95', 12: '0.00', 13: '0.34', 14: '89.57', 15: '0.00', 16: '0.00', 17: '79.15', 18: '0.00', 19: '68.28'}
Refined 2D  {'val_acc': 92.18358932039922, 'val_macc': 80.46455499883444, 'val_miou': 71.6021182077526}
{0: '89.69', 1: '90.01', 2: '89.83', 3: '0.00', 4: '77.12', 5: '40.89', 6: '80.20', 7: '83.02', 8: '89.53', 9: '0.00', 10: '16.10', 11: '86.48', 12: '0.00', 13: '11.89', 14: '90.43', 15: '0.00', 16: '0.00', 17: '80.60', 18: '0.00', 19: '76.64'}
